In [3]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
import os
family_group = "4.Pierinae"
annotated_genome_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/7.Timeless Exon Analysis/{family_group}/1.Blast_result"
species_list = os.listdir(annotated_genome_location)
if "desktop.ini" in species_list:
    species_list.remove("desktop.ini")
# species_list = ["Boloria_euphrosyne","Boloria_selene","Brenthis_daphne","Brenthis_hecate","Brenthis_ino","Dryadula_phaetusa","Dryas_iulia_moderata","Eueides_isabella","Fabriciana_adippe","Heliconius_charithonia","Heliconius_nattereri","Heliconius_sara","Philaethria_dido","Hestina_assimilis"]
# species_list = ["Limenitis_camilla"]
# species_list = ["Argynnis_bischoffii_washingtonia","Boloria_euphrosyne","Boloria_selene","Brenthis_daphne","Brenthis_hecate","Brenthis_ino","Dryadula_phaetusa","Dryas_iulia_moderata","Eueides_isabella","Fabriciana_adippe","Heliconius_charithonia","Heliconius_nattereri","Heliconius_sara","Philaethria_dido","Hestina_assimilis"]
# species_list = ["Bicyclus_anynana","Coenonympha_glycerion","Elymnias_hypermnestra","Erebia_aethiops","Erebia_ligea","Hipparchia_semele","Lasiommata_megera","Maniola_hyperantus","Maniola_jurtina","Melanargia_galathea","Oeneis_ivallda","Pararge_aegeria"]
# species_list = ["Erebia_ligea","Hipparchia_semele","Lasiommata_megera","Maniola_hyperantus","Maniola_jurtina","Melanargia_galathea","Oeneis_ivallda","Pararge_aegeria"]
# species_list = ["Vanessa_tameamea"]
for species in species_list:
    print(species)
    blast_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/7.Timeless Exon Analysis/{family_group}/1.Blast_result"
    
    list_of_files = os.listdir(f"{blast_location}/{species}")
    query_species_list = []
    coordinate_detail_dictionary = {}
    coordinate_info_dictionary = {}
    scaffold = ''
    for file in list_of_files:
        if file.endswith(".csv") and len(file.split("_")) >= 5 and not(file.endswith("_old.csv")):
#             print(file)
            if "." in file[:-4]:
                query_species = f"{file.split('_')[-2].split('.')[1]}_{file.split('_')[-1].split('.')[0]}"
            else:
                query_species = f"{file.split('_')[-2]}_{file.split('_')[-1].split('.')[0]}"
            query_species_list.append(query_species)
            coordinate_detail_dictionary[query_species] = {}
            with open(f"{blast_location}/{species}/{file}", 'r') as coordinate_file_open:
                coordinate_file_lines_list  =  coordinate_file_open.readlines()
            if scaffold == '':
                scaffold = coordinate_file_lines_list[1].split(",")[1]
    
            for i in range(1,len(coordinate_file_lines_list)):
                
                coordinate_file_lines_split = coordinate_file_lines_list[i].split(",")
                start,stop = coordinate_file_lines_split[2],coordinate_file_lines_split[3]
                exon = f"Exon_{i}"
                if exon not in coordinate_info_dictionary:
                    coordinate_info_dictionary[exon] = {}
                coordinate_info_dictionary[exon][query_species] = [start,stop]
                coordinate_detail_dictionary[query_species][exon] = coordinate_file_lines_list[i]
            
            # break
            # print(query_species)
            # print(file)
    # print(coordinate_info_dictionary)
    # assert False
    output = coordinate_file_lines_list[0]
    for exon,species_specific_coordinates in coordinate_info_dictionary.items():
        # print(exon,species_specific_coordinates)
        coordinates_list = []
        previous_coordinates = ""
        match = 1
        for query_species, coordinates in coordinate_info_dictionary[exon].items():
            coordinates_list.append([query_species,coordinates])
            if previous_coordinates != '':
                if coordinates != previous_coordinates:
                    match = 0
                    
            previous_coordinates = coordinates
       
        
        
        if match == 0:
            print(exon,"mismatch")
            print(coordinates_list)
                
            while True:
                choice = input("Proceed?")
                if choice[0].lower() == "y" or choice[0].lower() == "n":
                    break
            
            while True:
                if choice[0].lower() == "y":
                    alignment_detail_dictionary = {}
                    for query_species_name in query_species_list:
                        list_of_folders  = os.listdir(f"{blast_location}/{species}")
                        for folders in list_of_folders:
                            if folders.endswith(query_species_name):
                                # print()
                                
                                with open(f"{blast_location}/{species}/{folders}/{exon}/for_blast/new_query.txt", 'r') as query_file_open:
                                    # alignment_detail = query_file_open.readlines()[0]
                                    alignment_detail = query_file_open.readlines()[0].split("set")[1][:-1]
                                    alignment_detail_dictionary[query_species_name] = alignment_detail
                                with open (f"{blast_location}/{species}/{folders}/{exon}/for_alignment/{exon}_translated_genomic_sequence_{alignment_detail}.fa.hat2", 'r') as alignment_score_file:
                                    distance_score = float(alignment_score_file.readlines()[-1].rstrip())
                                try:
                                    with open (f"{blast_location}/{species}/{folders}/{exon}/for_blast/new_query_spliced.txt", 'r') as new_query_file:
                                        new_query = (new_query_file.readlines()[-1].rstrip())
                                except:
                                    new_query = "Issue with query"
                                print(query_species_name,exon, alignment_detail, distance_score,f"\n{new_query}")
                    while True:            
                        check_alignment = input("Check Alignment??")
                        if check_alignment[0].lower() == "y" or check_alignment[0].lower() == "n":
                            break
                    if check_alignment[0].lower() == "y":
                        for query_species_name in query_species_list:
                            list_of_folders  = os.listdir(f"{blast_location}/{species}")
                            for folders in list_of_folders:
                                if folders.endswith(query_species_name):
                                    with open (f"{blast_location}/{species}/{folders}/{exon}/for_alignment/alignment_clustal_{exon}_translated_genomic_sequence_{alignment_detail_dictionary[query_species_name]}.fa.txt", 'r') as clustal_alignment_file:
                                        clustal_alignment = clustal_alignment_file.readlines()
                                    print("".join(clustal_alignment))
                                    
                                # assert False
                    while True:
                        try:
                            species_number = (input(f"Choose Species 1 - {len(query_species_list)}"))
                            
                            
                            if species_number[0].lower() == 'n':
                                output +=  f'{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n'
                                break 
                            output += coordinate_detail_dictionary[query_species_list[int(species_number)-1]][exon]
                            break
                        except:
                            print("Retry")
                    break
                elif choice[0].lower() == "n":
                    output +=  f'{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n'
                    break
                
                
        else:
            print(exon, "Match")
            print(coordinates_list)
            output += coordinate_detail_dictionary[query_species_list[0]][exon]
        
       

        # assert False
    # print(coordinate_detail_dictionary)
    print(output)
    with open(f"{blast_location}/{species}/final_coordinates.csv", "w") as out_file:
        out_file.write(output)
    
    # 'Coenonympha_glycerion,OY979626.1,7015875,7015963,0,N,5.Bicyclus_anynana_XM_024088150.2_Frame_0_rightoh_2_query_Exon_1,1,29,29\n'

Anthocharis_cardamines
Exon_1 Match
[['Pieris_brassicae', ['7585053', '7585149']], ['Maniola_jurtina', ['7585053', '7585149']], ['Bicyclus_anynana', ['7585053', '7585149']]]
Exon_2 Match
[['Pieris_brassicae', ['7584086', '7584636']], ['Maniola_jurtina', ['7584086', '7584636']], ['Bicyclus_anynana', ['7584086', '7584636']]]
Exon_3 Match
[['Pieris_brassicae', ['7583313', '7583469']], ['Maniola_jurtina', ['7583313', '7583469']], ['Bicyclus_anynana', ['7583313', '7583469']]]
Exon_4 Match
[['Pieris_brassicae', ['7582631', '7582860']], ['Maniola_jurtina', ['7582631', '7582860']], ['Bicyclus_anynana', ['7582631', '7582860']]]
Exon_5 mismatch
[['Pieris_brassicae', ['7582156', '7582448']], ['Maniola_jurtina', ['7582156', '7582448']], ['Bicyclus_anynana', ['7582156', '7582433']]]
Proceed?y
Pieris_brassicae Exon_5 31_frame1 0.768 
NTISKQKQDQMEIQNHNEPTKLGEKIRASETSDCGYGTQIENQEVISTSSNEDELPTKKPVHQKPHNPKQRPNNKGRASVPLQDRKRKKMLKRGKAN
Maniola_jurtina Exon_5 31_frame1 0.703 
NTISKQKQDQMEIQNHNEPTKLGEKIRASE

Exon_1 Match
[['Pieris_brassicae', ['547532', '547628']], ['Maniola_jurtina', ['547532', '547628']], ['Bicyclus_anynana', ['547532', '547628']]]
Exon_2 Match
[['Pieris_brassicae', ['546457', '547004']], ['Maniola_jurtina', ['546457', '547004']], ['Bicyclus_anynana', ['546457', '547004']]]
Exon_3 Match
[['Pieris_brassicae', ['545170', '545323']], ['Maniola_jurtina', ['545170', '545323']], ['Bicyclus_anynana', ['545170', '545323']]]
Exon_4 Match
[['Pieris_brassicae', ['544512', '544741']], ['Maniola_jurtina', ['544512', '544741']], ['Bicyclus_anynana', ['544512', '544741']]]
Exon_5 mismatch
[['Pieris_brassicae', ['544049', '544338']], ['Maniola_jurtina', ['544049', '544338']], ['Bicyclus_anynana', ['544049', '544332']]]
Proceed?y
Pieris_brassicae Exon_5 35_frame1 0.782 
NEMMKGNEQMDSQSHDEPTGCDKKMTAETLDCGYSTQIENQEVISTSCNEDELPSKEPVHQKPNNPNQRPDNTVRGSVTFQERQRKKIGKRGKANI
Maniola_jurtina Exon_5 35_frame1 0.851 
NEMMKGNEQMDSQSHDEPTGCDKKMTAETLDCGYSTQIENQEVISTSCNEDELPSKEPVHQKPNNPNQRPDNTVRGSVTFQERQ

Exon_1 Match
[['Pieris_brassicae', ['5284038', '5284134']], ['Maniola_jurtina', ['5284038', '5284134']], ['Bicyclus_anynana', ['5284038', '5284134']]]
Exon_2 Match
[['Pieris_brassicae', ['5284207', '5284754']], ['Maniola_jurtina', ['5284207', '5284754']], ['Bicyclus_anynana', ['5284207', '5284754']]]
Exon_3 Match
[['Pieris_brassicae', ['5285108', '5285261']], ['Maniola_jurtina', ['5285108', '5285261']], ['Bicyclus_anynana', ['5285108', '5285261']]]
Exon_4 Match
[['Pieris_brassicae', ['5285546', '5285775']], ['Maniola_jurtina', ['5285546', '5285775']], ['Bicyclus_anynana', ['5285546', '5285775']]]
Exon_5 Match
[['Pieris_brassicae', ['5286049', '5286308']], ['Maniola_jurtina', ['5286049', '5286308']], ['Bicyclus_anynana', ['5286049', '5286308']]]
Exon_6 Match
[['Pieris_brassicae', ['5286380', '5286740']], ['Maniola_jurtina', ['5286380', '5286740']], ['Bicyclus_anynana', ['5286380', '5286740']]]
Exon_7 Match
[['Pieris_brassicae', ['5287041', '5287232']], ['Maniola_jurtina', ['5287041', '5

Choose Species 1 - 31
Exon_5 Match
[['Pieris_brassicae', ['14397144', '14397427']], ['Maniola_jurtina', ['14397144', '14397427']], ['Bicyclus_anynana', ['14397144', '14397427']]]
Exon_6 Match
[['Pieris_brassicae', ['14396567', '14396927']], ['Maniola_jurtina', ['14396567', '14396927']], ['Bicyclus_anynana', ['14396567', '14396927']]]
Exon_7 Match
[['Pieris_brassicae', ['14395760', '14395951']], ['Maniola_jurtina', ['14395760', '14395951']], ['Bicyclus_anynana', ['14395760', '14395951']]]
Exon_8 Match
[['Pieris_brassicae', ['14395059', '14395159']], ['Maniola_jurtina', ['14395059', '14395159']], ['Bicyclus_anynana', ['14395059', '14395159']]]
Exon_9 Match
[['Pieris_brassicae', ['14394722', '14394920']], ['Maniola_jurtina', ['14394722', '14394920']], ['Bicyclus_anynana', ['14394722', '14394920']]]
Exon_10 Match
[['Pieris_brassicae', ['14393960', '14394132']], ['Maniola_jurtina', ['14393960', '14394132']], ['Bicyclus_anynana', ['14393960', '14394132']]]
Exon_11 Match
[['Pieris_brassicae',

Proceed?y
Pieris_brassicae Exon_10 30_frame0 0.301 
DWSDLIEYVINTFIKKPYSLQRTASFRLDTESFDDDKILKDIIDNSKEDTTEDHLMLDVEDTD
Maniola_jurtina Exon_10 30_frame0 1.075 
DWSDLIEYVINTFIKKPYSLQRTASFRLDTESFDDDKILKDIIDNSKEDTTEDHL
Bicyclus_anynana Exon_10 30_frame0 1.323 
DWSDLIEYVINTFIKKPYSLQRTASFRLDTESFDDDKILKDIIDNSKEDTTEDHLMLDVEDTD
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Pieris_brassica --------DWSDLIEYVINTFIKKPHSLQRTASFRLDTESFDDDKVRKDLIDDSKE----
set30_frame0    CYFMHPFQDWSDLIEYVINTFIKKPYSLQRTASFRLDTESFDDDKILKDIIDNSKEDTTE
                        *****************:*******************: **:**:***    

Pieris_brassica --FMLDVKDTD-
set30_frame0    DHLMLDVEDTDM
                  :****:*** 

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Maniola_jurtina --------DWSDLIEYVINTFIKKPYKLQSSVQVRLDAEMFDNDTRLKEIL-MSAENTEE
set30_frame0    CYFMHPFQDWSDLIEYVINTFIKKPYSLQRTASFRLDTESFDDDKILKDIIDNSKEDTTE
                        ******************.** :...***:* **:*. **:*:  * *:* *

Mani

Choose Species 1 - 31
Exon_5 Match
[['Pieris_brassicae', ['1080292', '1080575']], ['Maniola_jurtina', ['1080292', '1080575']], ['Bicyclus_anynana', ['1080292', '1080575']]]
Exon_6 Match
[['Pieris_brassicae', ['1081092', '1081452']], ['Maniola_jurtina', ['1081092', '1081452']], ['Bicyclus_anynana', ['1081092', '1081452']]]
Exon_7 Match
[['Pieris_brassicae', ['1083052', '1083243']], ['Maniola_jurtina', ['1083052', '1083243']], ['Bicyclus_anynana', ['1083052', '1083243']]]
Exon_8 Match
[['Pieris_brassicae', ['1083959', '1084059']], ['Maniola_jurtina', ['1083959', '1084059']], ['Bicyclus_anynana', ['1083959', '1084059']]]
Exon_9 Match
[['Pieris_brassicae', ['1084232', '1084430']], ['Maniola_jurtina', ['1084232', '1084430']], ['Bicyclus_anynana', ['1084232', '1084430']]]
Exon_10 mismatch
[['Pieris_brassicae', ['1085216', '1085406']], ['Maniola_jurtina', ['1085216', '1085376']], ['Bicyclus_anynana', ['1085216', '1085406']]]
Proceed?y
Pieris_brassicae Exon_10 26_frame0 0.325 
DWSDLIEYVINTFIKK

Proceed?y
Pieris_brassicae Exon_4 29_frame1 0.546 
SNNSSPMLTSDATSDSSDTGRSGKTNEGSNSLNNSWDSTRNMSEQTFQSDPMQKKSCRIINNEQGENSRDKDIN
Maniola_jurtina Exon_4 29_frame1 1.256 
SNNSSPMLTSDATSDSSDTGRSGKTNEGSNSLNNSWDSTRNMSEQTFQSDPMQKKSCRIINNEQGENSRDKDINVRLF
Bicyclus_anynana Exon_4 29_frame1 1.247 
SNNSSPMLTSDATSDSSDTGRSGKTNEGSNSLNNSWDSTRNMSEQTFQSDPMQKKSCRIINNEQGENSRDKDINVRLF
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Pieris_brassica -------------------------------------------RDDSSPMLTSDPTSDSS
set29_frame1    KGSFGLPSQKFLCYQSETKIISIQTFLQLACVKNNKLQQFCTGSNNSSPMLTSDATSDSS
                                                            ::********.*****

Pieris_brassica DTGASGKTNGGSVSPDNSWVSTRNMGEQTFQLEPMQEKSCFIKNNEQGENSYEKDIN---
set29_frame1    DTGRSGKTNEGSNSLNNSWDSTRNMSEQTFQSDPMQKKSCRIINNEQGENSRDKDINVRL
                *** ***** ** * :*** *****.***** :***:*** * ******** :****   

Pieris_brassica -------------
set29_frame1    FVRFFMDKKKHLE
                             

CLUSTAL 

Exon_1 Match
[['Pieris_brassicae', ['6750840', '6750936']], ['Maniola_jurtina', ['6750840', '6750936']], ['Bicyclus_anynana', ['6750840', '6750936']]]
Exon_2 Match
[['Pieris_brassicae', ['6751743', '6752290']], ['Maniola_jurtina', ['6751743', '6752290']], ['Bicyclus_anynana', ['6751743', '6752290']]]
Exon_3 Match
[['Pieris_brassicae', ['6752647', '6752800']], ['Maniola_jurtina', ['6752647', '6752800']], ['Bicyclus_anynana', ['6752647', '6752800']]]
Exon_4 mismatch
[['Pieris_brassicae', ['6752985', '6753208']], ['Maniola_jurtina', ['6752985', '6753208']], ['Bicyclus_anynana', ['6752985', '6753220']]]
Proceed?y
Pieris_brassicae Exon_4 13_frame2 0.826 
SGDSSPMLTSEPISDYSDMGGSGKTNEGSISPNNSWDSNINMSEPIFQSEPMQEKSGHILNIGQRENSCDENIK
Maniola_jurtina Exon_4 13_frame2 1.397 
SGDSSPMLTSEPISDYSDMGGSGKTNEGSISPNNSWDSNINMSEPIFQSEPMQEKSGHILNIGQRENSCDENIK
Bicyclus_anynana Exon_4 13_frame2 1.368 
SGDSSPMLTSEPISDYSDMGGSGKTNEGSISPNNSWDSNINMSEPIFQSEPMQEKSGHILNIGQRENSCDENIKVRRF
Check Alignment??y
CLUSTAL forma

In [ ]:
1